In [27]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [28]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')
dataset.head()


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [29]:
#Customer ID and Surname would not be relevant as features
X_columns = dataset.columns.tolist()[2:12]
Y_columns = dataset.columns.tolist()[-1:]
print(X_columns)
print(Y_columns)

['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
['Exited']


In [30]:
X = dataset[X_columns].values 
Y = dataset[Y_columns].values

In [31]:
#We need to encode categorical variables such as geography and gender
from sklearn.preprocessing import LabelEncoder
X_column_transformer = LabelEncoder()
X[:, 1] = X_column_transformer.fit_transform(X[:, 1])

In [32]:
#Lets Encode gender now
X[:, 2] = X_column_transformer.fit_transform(X[:, 2])

We are treating countries with ordinal values(0 < 1 < 2) but they are incomparable.
To solve this we can use one hot encoding.
We will perform some standardization 

In [33]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    [
        ('Categorizer', ColumnTransformer(
            [
                ("Gender Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [2]),
                ("Geography Label Encoder", OneHotEncoder(categories = 'auto', drop = 'first'), [1])
            ], 
            remainder = 'passthrough', n_jobs = 1)),
        ('Normalizer', StandardScaler())
    ]
)

In [34]:
#Standardize the features
X = pipeline.fit_transform(X)

In [35]:
#Spilt the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [36]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [37]:
#Let us create the Neural Network
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [38]:
#Initialize ANN
classifier = Sequential()

In [49]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Initialize the model
classifier = Sequential()

# Add the input layer and first hidden layer
classifier.add(Input(shape=(X_train.shape[1],)))  # Define the input shape
classifier.add(Dense(6, activation='relu'))  # First hidden layer
classifier.add(Dropout(rate=0.1))  # Dropout layer


In [50]:
#Add second layer
classifier.add(Dense(6, activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [51]:
#Add output layer
classifier.add(Dense(1, activation = 'sigmoid'))

In [52]:
#Let us take a look at our network
classifier.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 6)              │            42 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             7 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121 (484.00 B)

 Trainable params: 121 (484.00 B)

 Non-trainable params: 0 (0.00 B)

In [53]:
#Optimize the weights
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [54]:
#Fitting the Neural Network
history = classifier.fit(X_train, y_train, batch_size = 32, epochs = 200, validation_split = 0.1, verbose = 2)

Epoch 1/200
225/225 - 2s - 10ms/step - accuracy: 0.5014 - loss: 0.7641 - val_accuracy: 0.7700 - val_loss: 0.6075
Epoch 2/200
225/225 - 0s - 1ms/step - accuracy: 0.7790 - loss: 0.5665 - val_accuracy: 0.7975 - val_loss: 0.5203
Epoch 3/200
225/225 - 0s - 2ms/step - accuracy: 0.7968 - loss: 0.5097 - val_accuracy: 0.8050 - val_loss: 0.4797
Epoch 4/200
225/225 - 0s - 2ms/step - accuracy: 0.8018 - loss: 0.4862 - val_accuracy: 0.8075 - val_loss: 0.4558
Epoch 5/200
225/225 - 0s - 2ms/step - accuracy: 0.8081 - loss: 0.4668 - val_accuracy: 0.8150 - val_loss: 0.4378
Epoch 6/200
225/225 - 0s - 2ms/step - accuracy: 0.8103 - loss: 0.4560 - val_accuracy: 0.8200 - val_loss: 0.4233
Epoch 7/200
225/225 - 0s - 2ms/step - accuracy: 0.8122 - loss: 0.4477 - val_accuracy: 0.8250 - val_loss: 0.4132
Epoch 8/200
225/225 - 0s - 2ms/step - accuracy: 0.8183 - loss: 0.4380 - val_accuracy: 0.8275 - val_loss: 0.4025
Epoch 9/200
225/225 - 0s - 2ms/step - accuracy: 0.8163 - loss: 0.4337 - val_accuracy: 0.8238 - val_loss

In [55]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[0.17874081]
 [0.21996188]
 [0.19070846]
 [0.1296877 ]
 [0.08603524]]


In [56]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [57]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1548   47]
 [ 232  173]]


In [58]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.05 % of data was classified correctly
